In [2]:
import numpy as np

# numerical_flux.py

In [3]:
A = np.arange(25).reshape(5,5)
A = A[...]
B = np.stack((A,A,A), axis=0)
print(B.shape)
# print(B[:,:,0])
# B = np.transpose(B, (0,1,2))

axes = [1,0,2]
B = np.transpose(B, np.argsort(axes))
# print(B)
# print(B[:,:,1:-1])

# axes = [2,1,0]
B = np.transpose(B, np.argsort(axes))
# print(B)
# print(B[:,:,1:-1])

B = np.moveaxis(B, 1, 2)
# print(B)

B = np.moveaxis(B, 1, 2)
# print(B)
# print(B.ndim)
# B = np.moveaxis(B,0,-1)
print(B)

# print(B.reshape(-1))

cnt, cntx, cnty, cntz = 0, 0, 0, 0
nsnc = B.reshape(-1).shape[0]

iczn, icyn, icxn = B.shape

for idx in range(nsnc):
    
#     print("cnt=%i, cntx=%i, cnty=%i, cntz=%i, cnty_r=%i" %(cnt, cntx, cnty, cntz, cnty % (icxn * icyn)))
    
    cntx += 1
    if cntx % icxn == 0:
        cnty += 1
        cntx = 0
        
        if cnty % icyn == 0:
            cntz += 1
            cnty = 0
            
    cnt += 1

(3, 5, 5)
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]

 [[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]
  [20 21 22 23 24]]]


In [4]:
import numba as nb
from scipy import signal

@nb.jit(nopython=True, cache=True)
def dict_func(lst,a):
    plst = lst[1:-1,1:-1,:]
    leftx = plst[:,:,:-1]
    rightx = plst[:,:,1:]

    x_fluxes = (rightx - leftx)
#     print(x_fluxes.shape)

    plst = lst[1:-1,:,1:-1]
    lefty = plst[:,:-1,:]
    righty = plst[:,1:,:]

    y_fluxes = (righty - lefty)
#     print(y_fluxes.shape)

    plst = lst[:,1:-1,1:-1]
    leftz = plst[:-1,:,:]
    rightz = plst[1:,:,:]

    z_fluxes = (rightz - leftz)
#     print(z_fluxes.shape)
    p = - x_fluxes[:,:,:-1] + x_fluxes[:,:,1:] \
        - y_fluxes[:,:-1,:] + y_fluxes[:,1:,:] - z_fluxes[:-1,:,:] + z_fluxes[1:,:,:]
#     lst = lst / 16
    return p
    
# np.random.seed(555)
shp = lst.shape

# # p = np.zeros_like(lst)
# plst = lst.tolist()
# plst = lst[:,:,-1][...,None] + lst[:,:,:] + lst[:,:,0][...,None]
lst = np.random.randint(15,size=(64,35,64))
lst = np.pad(lst,((0,0),(0,0),(1,1)),mode='wrap')
lst = np.pad(lst,((0,0),(1,1),(0,0)),mode='wrap')
lst = np.pad(lst,((1,1),(0,0),(0,0)),mode='constant')

for i in range(200):
    p = dict_func(lst,a)

print(p.shape)

NameError: name 'lst' is not defined

In [ ]:
lst = np.random.randint(15,size=(10,20,30))

@nb.jit(nopython=True, cache=True)
def tpose(lst):
    lst = np.moveaxis(lst,0,-1)
    print(lst.shape)

# tpose(lst)
lst0 = lst.reshape(20,30,10)
lst = np.moveaxis(lst,0,-1)


grid = np.arange(27).reshape(3,3,3)
print(grid.shape)

In [111]:
sz = 25

lst = np.random.randint(15,size=(70,38,70))
bx,by,bz = lst.shape[0],lst.shape[1],lst.shape[2]
print(bx,by,bz)

def op(v):
    v = v.reshape(bx,by)
    v = np.pad(v[2:-2,2:-2],(2,2),mode='constant')
    lap = np.diag(np.ones(v.shape[0]))
    lap = np.dot(lap,v)

    return lap

import h5py

i0 = (slice(0,-1),slice(0,-1),slice(0,-1))
i1 = (slice(1,-1),slice(1,-1),slice(1,-1))
i2 = (slice(2,-2),slice(2,-2),slice(2,-2))

hplusx = np.zeros_like(lst)
hplusy = np.zeros_like(lst)
hplusz = np.zeros_like(lst)
hcenter = np.zeros_like(lst)

hplusx[i0] = h5py.File('/home/ray/git-projects/RKLM_Reference/wplusx.h5')['Data-Set-2'][...]
hplusy[i0] = h5py.File('/home/ray/git-projects/RKLM_Reference/wplusy.h5')['Data-Set-2'][...]
hplusz[i0] = h5py.File('/home/ray/git-projects/RKLM_Reference/wplusz.h5')['Data-Set-2'][...]
hcenter[...] = h5py.File('/home/ray/git-projects/RKLM_Reference/hcenter.h5')['Data-Set-2'][...]

hcenter = hcenter[i2]
hplusx = hplusx[i1]
hplusy = hplusy[i1]
hplusz = hplusz[i1]

# print("hcs= ", hcenter.shape)

rhs = h5py.File('/home/ray/git-projects/RKLM_Reference/rhs.h5')['Data-Set-2'][...]

hplusx = hplusx[i0]
hplusy = hplusy[i0]
hplusz = hplusz[i0]

oodx2, oody2, oodz2 = 1.,1.,1.
coeff = 1./5

# @nb.jit(nopython=True, cache=True)
def lap3D(p):

    shpx, shpy, shpz = 68, 36, 68
#     lefts = [slice(None,None,-1),slice(None,-1,None),slice(-1,None,None)]
    lefts = [(slice(0,-1),slice(0,None),slice(0,None)),
             (slice(0,None,),slice(0,-1),slice(0,None)),
             (slice(0,None,),slice(0,None),slice(0,-1))
            ]
    
    rights = [(slice(1,None),slice(0,None),slice(0,None)),
              (slice(0,None),slice(1,None),slice(0,None)),
              (slice(0,None),slice(0,None),slice(1,None))
             ]
    
    toplefts = [(slice(0,None),slice(0,-1),slice(0,-1)),
                (slice(0,-1),slice(0,None),slice(0,-1)),
                (slice(0,-1),slice(0,-1),slice(0,None))
             ]
    toprights = [(slice(0,None),slice(0,-1),slice(1,None)),
                 (slice(1,None),slice(0,None),slice(0,-1)),
                 (slice(0,-1),slice(1,None),slice(0,None))
                ]
    
    botlefts = [(slice(0,None),slice(1,None),slice(0,-1)),
                (slice(0,-1),slice(0,None),slice(1,None)),
                (slice(1,None),slice(0,-1),slice(0,None))
               ]
    botrights = [(slice(0,None),slice(1,None),slice(1,None)),
                 (slice(1,None),slice(0,None),slice(1,None)),
                 (slice(1,None),slice(1,None),slice(0,None))
                ]
    
    corners = [toplefts,toprights,botlefts,botrights]
#     left = (slice(None,-1),slice(0,None),slice(0,None))
    p = p.reshape(shpx,shpy,shpz)
    lap = np.zeros_like(p)
    print(p.shape)
    
    leftz = p[:,:,:-1]
    rightz = p[:,:,1:]
    
    z_fluxes = (rightz - leftz)

    lefty = p[:,:-1,:]
    righty = p[:,1:,:]

    y_fluxes = (righty - lefty)

    leftx = p[:-1,:,:]
    rightx = p[1:,:,:]

    x_fluxes = (rightx - leftx)

    x_flx = x_fluxes[toplefts[0]] + x_fluxes[toprights[0]] + x_fluxes[botlefts[0]] + x_fluxes[botrights[0]]
    y_flx = y_fluxes[toplefts[1]] + y_fluxes[toprights[1]] + y_fluxes[botlefts[1]] + y_fluxes[botrights[1]]
    z_flx = z_fluxes[toplefts[2]] + z_fluxes[toprights[2]] + z_fluxes[botlefts[2]] + z_fluxes[botrights[2]]

#     print(x_flx.shape)
#     print(y_flx.shape)
#     print(z_flx.shape)
    x_flx = hplusx * x_flx
    x_flxm = x_flx[:-1,:,:]
    x_flxm = x_flxm[toplefts[0]] + x_flxm[toprights[0]] + x_flxm[botlefts[0]] + x_flxm[botrights[0]]
    x_flxp = x_flx[1:,:,:]
    x_flxp = x_flxp[toplefts[0]] + x_flxp[toprights[0]] + x_flxp[botlefts[0]] + x_flxp[botrights[0]]

    y_flx *= hplusy
    y_flxm = y_flx[:,:-1,:]
    y_flxm = y_flxm[toplefts[1]] + y_flxm[toprights[1]] + y_flxm[botlefts[1]] + y_flxm[botrights[1]]
    y_flxp = y_flx[:,1:,:]
    y_flxp = y_flxp[toplefts[1]] + y_flxp[toprights[1]] + y_flxp[botlefts[1]] + y_flxp[botrights[1]]

    z_flx *= hplusz
    z_flxm = z_flx[:,:,:-1]
    z_flxm = z_flxm[toplefts[2]] + z_flxm[toprights[2]] + z_flxm[botlefts[2]] + z_flxm[botrights[2]]
    z_flxp = z_flx[:,:,1:]
    z_flxp = z_flxp[toplefts[2]] + z_flxp[toprights[2]] + z_flxp[botlefts[2]] + z_flxp[botrights[2]]
    
    print(x_flxm.shape, x_flxp.shape)
    print(y_flxm.shape, y_flxp.shape)
    print(z_flxm.shape, z_flxp.shape)
    
    lap[1:-1,1:-1,1:-1] = oodx2 * coeff * (+x_flxm - x_flxp) + \
                      oody2 * coeff * (+y_flxm - y_flxp) + \
                      oodz2 * coeff * (+z_flxm - z_flxp) + \
                      hcenter * p[1:-1,1:-1,1:-1]

#     lap[1:-1,1:-1,1:-1] = hplusx * oodx2 * coeff * (- x_fluxes[:,:,:-1] + x_fluxes[:,:,1:]) \
#         + hplusy * oody2 * coeff * (- y_fluxes[:,:-1,:] + y_fluxes[:,1:,:]) \
#         + hplusz * oodz2 * coeff * (- z_fluxes[:-1,:,:] + z_fluxes[1:,:,:]) \
#         + hcenter * p[1:-1,1:-1,1:-1]
    return lap

from scipy.sparse import linalg as la

b = np.arange(sz).reshape(5,5)
b = np.pad(b,(2,2),mode='wrap')

b = b.reshape(-1)

def wrap(xk):
    xk = xk.reshape(bx,by)
    return xk

lst = lst[i1]
p = lst.reshape(-1)
pshp = p.shape[0]
print(p.shape)
D = la.LinearOperator((pshp,pshp),lap3D)
x, _ = la.bicgstab(D,p)
print(x.reshape(bx,by))

70 38 70
(166464,)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 3

(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(66, 34, 66) (66, 34, 66)
(68, 36, 68)

KeyboardInterrupt: 

In [104]:
lst = np.random.randint(15,size=(30,40,50))

from scipy import signal

print(lst.shape)

kernel_x = np.ones((1,2,2))
kernel_y = np.ones((2,1,2))
kernel_z = np.ones((2,2,1))
print(kernel_x.shape)
lst_pp = signal.fftconvolve(lst, kernel_x, mode='valid')
print(lst_pp.shape)

print(lst_pp[(slice(1,None),slice(0,None),slice(0,None))].shape)

(30, 40, 50)
(1, 2, 2)
(30, 39, 49)
(29, 39, 49)


In [22]:
y0 = [0.9869944041776586,0.9934759202807376,1.,1.0065669228474292,1.0131769701705426,1.0198304251644446,1.0265275728839605,1.0332687002558483,1.0400540960910918,1.0468840510972748,1.0537588578910349,1.0606788110106005,1.06764420692841,1.0746553440638136,1.0817125227958582,1.0888160454761566,1.0959662164418416,1.103163342028604,1.110407730583818,1.117699692479751,1.125039540126861,1.1324275879871812,1.1398641525877935,1.147349552534388,1.1548841085249135,1.1624681433633188,1.17010198197338,1.1777859514126232,1.185520380886336,1.1933056017616714,1.2011419475818454,1.2090297540804265,1.216969359195719,1.2249611030852425,1.233005328140305,1.2411023790006717,1.24925260256933,1.257456348027353]

y0 = np.array(y0)
print(y0)
dy = 0.5454545454545454

yy = 2.0 * (y0[1:] - y0[:-1]) / (y0[1:] + y0[:-1]) / dy
print(yy)
# [0.011999957157208304 0.011999957157208543 0.011999957157208061
#  0.01199995715720842  0.011999957157208434 0.01199995715720812
#  0.011999957157208425 0.011999957157208517 0.011999957157208415
#  0.011999957157208255 0.011999957157208233 0.011999957157208354
#  0.011999957157208375 0.01199995715720836  0.011999957157208242
#  0.011999957157208346 0.011999957157208576 0.011999957157208202
#  0.01199995715720828  0.011999957157208441 0.011999957157208549
#  0.011999957157207973 0.011999957157208512 0.011999957157208266
#  0.011999957157208328 0.011999957157208327 0.011999957157208373
#  0.011999957157208335 0.011999957157208552 0.011999957157208278
#  0.011999957157208247 0.011999957157208238 0.011999957157208424]

[0.9869944  0.99347592 1.         1.00656692 1.01317697 1.01983043
 1.02652757 1.0332687  1.0400541  1.04688405 1.05375886 1.06067881
 1.06764421 1.07465534 1.08171252 1.08881605 1.09596622 1.10316334
 1.11040773 1.11769969 1.12503954 1.13242759 1.13986415 1.14734955
 1.15488411 1.16246814 1.17010198 1.17778595 1.18552038 1.1933056
 1.20114195 1.20902975 1.21696936 1.2249611  1.23300533 1.24110238
 1.2492526  1.25745635]
[0.01199996 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996
 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996
 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996
 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996
 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996
 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996 0.01199996
 0.01199996]


In [61]:
def stencil_27pt(elem,node,,ud,diag_inv):

    i0 = (slice(0,-1),slice(0,-1),slice(0,-1))
    i1 = (slice(1,-1),slice(1,-1),slice(1,-1))
    i2 = (slice(2,-2),slice(2,-2),slice(2,-2))

    ndim = elem.ndim
    periodicity = np.empty(ndim, dtype='int')
    for dim in range(ndim):
        periodicity[dim] = ud.bdry_type[dim] == BdryType.PERIODIC

    kernel_x = np.ones((1,2,2))
    kernel_y = np.ones((2,1,2))
    kernel_z = np.ones((2,2,1))

    hplusx = mpv.wplus[0][i0][i1]
    hplusy = mpv.wplus[1][i0][i1]
    hplusz = mpv.wplus[2][i0][i1]

    # print(hplusx[:,0,:],hplusx[:,-1,:])

    hplusx = signal.fftconvolve(hplusx,kernel_x,mode='valid')
    hplusy = signal.fftconvolve(hplusy,kernel_y,mode='valid')
    hplusz = signal.fftconvolve(hplusz,kernel_z,mode='valid')

    # generalise to support wall along any axis, and not just the y-axis.
    # hplusx[:,0,:],hplusx[:,-1,:] = 0.0, 0.0
    # hplusy[:,0,:],hplusy[:,-1,:] = 0.0, 0.0
    # hplusz[:,0,:],hplusz[:,-1,:] = 0.0, 0.0

    # print(hplusx.shape, hplusy.shape, hplusz.shape)

    hcenter = mpv.wcenter[i2]
    diag_inv = diag_inv[i1]

    return lambda p : lap3D(p, hplusx, hplusy, hplusz, hcenter, oodx2, oody2, oodz2, periodicity, diag_inv)
    # return lambda p : lap3D(p)

@nb.jit(nopython=True, cache=False)
# def lap3D(p0):
def lap3D(p0, hplusx, hplusy, hplusz, hcenter, oodx2, oody2, oodz2, periodicity, diag_inv):
    # p = p0.reshape(shp[0],shp[1],shp[2])
    # p = p0.reshape(hcenter.shape)
    shx, shy, shz = hcenter.shape
    # p = p0.reshape(shx+2,shy+2,shz+2)
    p = p0.reshape(shz+2,shy+2,shx+2)
    # p = p0.reshape(66,34,66)

    coeff = 1./16
    # hplusx, hplusy, hplusz = 1.0, 1.0, 1.0
    # hcenter = 1.0
    # oodx2, oody2, oodz2 = 1.0, 1.0, 1.0
    lap = np.zeros_like(p)

    lefts = [(slice(0,-1),slice(0,None),slice(0,None)),
             (slice(0,None,),slice(0,-1),slice(0,None)),
             (slice(0,None,),slice(0,None),slice(0,-1))
            ]
    
    rights = [(slice(1,None),slice(0,None),slice(0,None)),
              (slice(0,None),slice(1,None),slice(0,None)),
              (slice(0,None),slice(0,None),slice(1,None))
             ]
    
    toplefts = [(slice(0,None),slice(0,-1),slice(0,-1)),
                (slice(0,-1),slice(0,None),slice(0,-1)),
                (slice(0,-1),slice(0,-1),slice(0,None))
             ]
    toprights = [(slice(0,None),slice(0,-1),slice(1,None)),
                 (slice(1,None),slice(0,None),slice(0,-1)),
                 (slice(0,-1),slice(1,None),slice(0,None))
                ]
    
    botlefts = [(slice(0,None),slice(1,None),slice(0,-1)),
                (slice(0,-1),slice(0,None),slice(1,None)),
                (slice(1,None),slice(0,-1),slice(0,None))
               ]
    botrights = [(slice(0,None),slice(1,None),slice(1,None)),
                 (slice(1,None),slice(0,None),slice(1,None)),
                 (slice(1,None),slice(1,None),slice(0,None))
                ]

    corners = [toplefts,toprights,botlefts,botrights]
    cnt = 0
    for bc in periodicity:
        if bc == True and cnt == 0:
            # tmp = p[1,:,:]
            p[0,:,:] = p[-2,:,:]
            p[-1,:,:] = p[1,:,:]
            # p[1,:,:] = p[-2,:,:]
            # p[-2,:,:] = tmp
        elif bc == True and cnt == 2:
            # tmp = p[:,:,1]
            p[:,:,0] = p[:,:,-2]
            p[:,:,-1] = p[:,:,1]
            # p[:,:,1] = p[:,:,-2]
            # p[:,:,-2] = tmp
        cnt += 1
    
    # pinner = p[1:-1,1:-1,:]
    leftz = p[:,:,:-1]
    rightz = p[:,:,1:]
    
    z_fluxes = (rightz - leftz)

    # pinner = p[1:-1,:,1:-1]
    lefty = p[:,:-1,:]
    righty = p[:,1:,:]

    y_fluxes = (righty - lefty)

    # pinner = p[:,1:-1,1:-1]
    leftx = p[:-1,:,:]
    rightx = p[1:,:,:]

    x_fluxes = (rightx - leftx)

    x_flx = x_fluxes[toplefts[0]] + x_fluxes[toprights[0]] + x_fluxes[botlefts[0]] + x_fluxes[botrights[0]]
    y_flx = y_fluxes[toplefts[1]] + y_fluxes[toprights[1]] + y_fluxes[botlefts[1]] + y_fluxes[botrights[1]]
    z_flx = z_fluxes[toplefts[2]] + z_fluxes[toprights[2]] + z_fluxes[botlefts[2]] + z_fluxes[botrights[2]]

    x_flx *= hplusx
    x_flxm = x_flx[:-1,:,:]
    x_flxp = x_flx[1:,:,:]

    y_flx *= hplusy
    y_flxm = y_flx[:,:-1,:]
    y_flxp = y_flx[:,1:,:]

    z_flx *= hplusz
    z_flxm = z_flx[:,:,:-1]
    z_flxp = z_flx[:,:,1:]

    lap[1:-1,1:-1,1:-1] = oodx2 * coeff * (+x_flxm - x_flxp) + \
                          oody2 * coeff * (+y_flxm - y_flxp) + \
                          oodz2 * coeff * (+z_flxm - z_flxp) + \
                          hcenter * p[1:-1,1:-1,1:-1]

    # lap[1:-1,1:-1,1:-1] = oodx2 * coeff * (-(hplusx[:,:,:-1] * x_fluxes[:,:,:-1]) + (hplusx[:,:,1:] * x_fluxes[:,:,1:])) \
    #     + oody2 * coeff * (-(hplusy[:,:-1,:] * y_fluxes[:,:-1,:]) + (hplusy[:,1:,:] * y_fluxes[:,1:,:])) \
    #     + oodz2 * coeff * (-(hplusx[:-1,:,:] * z_fluxes[:-1,:,:]) + (hplusx[1:,:,:] * z_fluxes[1:,:,:])) \
    #     + hcenter * p[1:-1,1:-1,1:-1]

    lap = lap * diag_inv

    return lap

diag_inv = precon_diag_prepare(mpv, elem, node, ud)
rhs *= diag_inv

p2 = rhs[1:-1,1:-1,1:-]
# lap = stencil_27pt(elem,node,mpv,ud,diag_inv)

sh = p2.reshape(-1).shape[0]
lap = LinearOperator((sh,sh),lap)